In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import bz2
from glob import glob
def restore_df(path):
    with pd.HDFStore(path, mode = 'r') as store:
        df = store.get('game')
        df.attrs = store.get_storer('game').attrs.my_attribute
        return df

In [10]:
italy_wales = restore_df("../input/socceranalytics/velocities/velocities/italyvwales.h5.h5")

In [84]:
players = []
for player_id, player_name in italy_wales.attrs['player_map'].items():
    players.append(player_name)

In [85]:
df = pd.read_csv('../input/fifa-player-stats-database/FIFA21_official_data.csv')
df.head()

In [86]:
df.info()

In [117]:
to_drop = ["Special", "Club Logo", "Flag", "Photo", "Real Face", "Skill Moves"]
df = df.drop(to_drop, axis= 1)

In [118]:
 def leven(x, y):
    """
    String Levenshtein distance (edit distance)
    From https://devrescue.com/levenshtein-distance-in-python/
    """
    n = len(x)
    m = len(y)
    
    x_l = x.lower()
    y_l = y.lower()

    A = [[i + j for j in range(m + 1)] for i in range(n + 1)]

    for i in range(n):
        for j in range(m):
            A[i + 1][j + 1] = min(A[i][j + 1] + 1,              # insert
                                  A[i + 1][j] + 1,              # delete
                                  A[i][j] + int(x_l[i] != y_l[j]))  # replace

    return A[n][m] / max(len(x), len(y))

In [119]:
last_names = [player.split(" ")[-1] for player in players]

In [120]:
df['possible_match'] = df.Name.apply(lambda x: x.split(" ")[-1] in last_names)

In [121]:
df_matches = df[df['possible_match'] == True]

In [122]:
distances = dict()
for player in players:
    distances[player] = []
    initials = player.split(" ")[0][0] + " " + player.split(" ")[-1]
    for entry in df_matches.Name.to_numpy():
        distances[player].append((leven(initials.strip(), entry.strip()), entry))

In [123]:
fifa_map = dict()
fifa_names = []
for key, poss in distances.items():
    name = sorted(poss, key=lambda x: x[0])[0][1]
    fifa_names.append(name)
    fifa_map[key] = name

In [124]:
len(players)

In [125]:
len(fifa_names)

In [126]:
#some players are double because same name
our_players = df[df.Name.isin(fifa_names) & (df.Nationality.isin(["Italy", "Wales"]))].sort_values(by = "Name")
#our_players = df[df.Name.isin(fifa_names)].sort_values(by = "Name")

In [147]:
df.Value.iloc[0][1:][:-1]

In [149]:
def money_convert(x):
    x = x[1:]
    mult = x[-1]
    x = x[:-1]
    x = float(x)
    if mult == "K":
        x = x*1e3
    if mult == "M":
        x = x*1e6
    return x

In [127]:
#one player missing
our_players

In [151]:
our_players['Value'] = our_players['Value'].apply(money_convert)
our_players['Wage'] = our_players['Wage'].apply(money_convert)
italy_fifa = our_players[our_players.Nationality == "Italy"]
wales_fifa = our_players[our_players.Nationality == "Wales"]

In [153]:
our_players.groupby('Nationality').mean()[['Value', 'Wage', 'Overall']]

In [133]:
italy_fifa['Overall'].describe()

In [134]:
wales_fifa['Overall'].describe()

In [136]:
italy_fifa['Preferred Foot'].value_counts()

In [137]:
wales_fifa['Preferred Foot'].value_counts()

In [154]:
italy_fifa['Value'].describe()

In [156]:
wales_fifa['Value'].describe()